# Plotting codes that need to be moved to Bokeh: Violin Plots

This is the plotting code without any of the code to generate the inputs. 


In [1]:
import bisect
import numpy as np
np.random.seed(42)
import os
import pandas as pd
import pickle
# from sklearn.neighbors.kde import KernelDensity
import scipy.stats as sps
# import seaborn as sns
import sys

epsilon = sys.float_info.epsilon

In [3]:
import matplotlib as mpl
# print(mpl.rcParams.items)
mpl.use('PS')
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["mathtext.fontset"] = "dejavuserif"
mpl.rcParams['font.serif'] = 'DejaVu Serif'
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['savefig.dpi'] = 250
mpl.rcParams['figure.dpi'] = 250
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'
import matplotlib.pyplot as plt
%matplotlib inline
# print(mpl.rcParams.items)
from matplotlib.colors import LogNorm

import pylab
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = plt.get_cmap('hot_r')
fave_cmap = truncate_colormap(cmap, 0.35, 1.0)


In [5]:
#import proclam
#from proclam import *

In [2]:
label_dict = {90: 'SNIa',
              67: 'SNIa-91bg',
              52: 'SNIax',
              42: 'SNII',
              62: 'SNIbc',
              95: 'SLSN-I',
              15: 'TDE',
              64: 'KN',
              88: 'AGN',
              92: 'RRL',
              65: 'M-dwarf',
              16: 'EB',
              53: 'Mira',
              6: r'$\mu$Lens-Single'}

true_labels = label_dict.copy()
true_labels[991] = r'$\mu$Lens-Binary'
true_labels[992] = 'ILOT'
true_labels[993] = 'CaRT'
true_labels[994] = 'PISN'
true_labels[995] = r'$mu$Lens-String'

sub_labels = label_dict.copy()
sub_labels[99] = 'Other'

In [ ]:
def violins(contestant, one_target, index, class_dict, color='r'):
    all_prob = pd.read_csv(os.path.join('submissions/'+contestant, 'probvecs'+str(one_target)+'true.csv'), index_col=index)
    n_true = len(all_prob)
    stretchfact = 2.5
    loc = 0
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.set_title(str(n_true)+' true '+str(true_labels[one_target])+' by '+contestant)
    ticklabels = []
    maps = all_prob.idxmax(axis=1)
    counts = maps.value_counts()
#     print(counts.keys())
    for cl_est in class_dict.keys():
        if str(cl_est) in counts.keys():
            nmax = str(counts[str(cl_est)])#str(np.around(counts[str(cl_est)]/float(n_true), 2))
        else:
            nmax = '0'
        if cl_est == one_target or (one_target == 99 and str(cl_est)[:2] == '99'):
            highlight = color
            textcol = 'k'
            alp = 0.75
        else:
            highlight = 'k'
            textcol = color
            alp = 0.25
        data = np.exp(np.genfromtxt(os.path.join('submissions/'+contestant, 'violin'+str(one_target)+'true'+str(cl_est)+'pred.txt')))
        if not np.all(data == 1.):
            data = data / np.max(data)
        else:
            data = np.zeros_like(data)
        wheremax = np.argmax(data)
        ax.fill_betweenx(positions, stretchfact*loc - data, stretchfact*loc + data, alpha=alp, color=highlight, linewidth=0.1)
        if wheremax < len(positions)/4:
            whereprint = 'bottom'
        elif wheremax > 3 * len(positions) / 4:
            whereprint = 'top'
        else:
            whereprint = 'center'
        ax.text(stretchfact*loc, positions[wheremax], nmax, fontsize=12, color=textcol, ha='center', va=whereprint, rotation=90)#fontweight='bold', 
        loc += 1
        ticklabels.append(sub_labels[cl_est])
    ax.set_xticks(stretchfact * np.arange(loc))
    ax.set_xticklabels(ticklabels, rotation=45, ha="right")
    ax.set_xlabel('predicted class')
    ax.set_ylabel('probability')
    fig.savefig(os.path.join('submissions/'+contestant, 'violin_'+true_labels[one_target]+'.png'))

In [ ]:
color_dict = {'1_Kyle': '#6344EE', '2_MikeSilogram': '#E873AB', '3_MajorTom': '#FFB81F', 'validation': '#30362F'}

In [ ]:
contestants = ['1_Kyle', '2_MikeSilogram', '3_MajorTom']
all_contestants = ['validation'] + contestants

In [ ]:
for contestant in contestants:
    for one_target in truth.true_target.unique():#label_dict:
#         violins(contestant, one_target, 'objids', label_dict, color='#00A878')
        violins(contestant, one_target, 'object_id', sub_labels, color=color_dict[contestant])

In [ ]:
    for one_target in label_dict.keys():#truth.true_target.unique():#label_dict:
#         violins(contestant, one_target, 'objids', label_dict, color='#00A878')
        violins(contestant, one_target, 'object_id', label_dict, color=color_dict[contestant])